In [ ]:
from PIL import Image
import fiftyone as fo
import json
from tqdm import tqdm
import pandas as pd

# 变量定义
datasets = {}


for data_type in ['test_synth', 'test_real_nominal', 'test_real_edge']:

    print(f'正在构建: {data_type}')
    extension = ".jpeg" if "synth" in data_type else ".png"
    path_predictions = f"runs/detect/yolov8n_train_val/val_{data_type}/predictions.json"

    # 构建数据集
    dataset = fo.load_dataset(f"lard_{data_type}")

    # 获取数据集字段架构
    field_schema = dataset.get_field_schema()
    # for field_name, field_type in field_schema.items():
    #     print(f"{field_name}: {field_type}")

    # 删除字段
    try:
        dataset.delete_sample_fields(
            ["predictions", "eval_tp", "eval_fp", "eval_fn"])
    except:
        pass

    # 类别信息
    classes = dataset.default_classes

    # 加载 JSON 预测文件
    with open(path_predictions, "r") as f:
        predictions_data = json.load(f)

    # 遍历追加预测结果
    for sample in tqdm(dataset, desc="正在加载预测结果"):

        filename = sample.filepath.split('/')[-1].split(extension)[0]

        image = Image.open(sample.filepath)
        width, height = image.size

        predictions_for_sample = [
            rec for rec in predictions_data if rec['image_id'] == filename]

        detections = []
        for pred in predictions_for_sample:

            category_id = pred['category_id']
            bbox = pred['bbox']
            score = pred['score']

            x1, y1, w, h = bbox
            rel_box = [x1 / width, y1 / height, w / width, h / height]

            detections.append(
                fo.Detection(
                    label=classes[category_id],
                    bounding_box=rel_box,
                    confidence=score
                )
            )

        sample["predictions"] = fo.Detections(detections=detections)
        sample.save()

    print("正在执行评估")
    # 评估
    results = dataset.evaluate_detections(
        "predictions",
        gt_field="ground_truth_detections",
        method="coco",
        eval_key="eval",
        compute_mAP=True,
    )

    # 打印报告
    results.print_report()

    # 打印指标TP/FP/FN
    print(f"TP: {dataset.sum('eval_tp')}")
    print(f"FP: {dataset.sum('eval_fp')}")
    print(f"FN: {dataset.sum('eval_fn')}")
    print(f"mAP: {results.mAP():.3f}")
    display(pd.DataFrame(results.metrics(), index=[0]).round(3))

    datasets[data_type] = dataset

In [ ]:
# 绘图
results.plot_pr_curves()

In [ ]:
results.plot_confusion_matrix()

In [ ]:
fo.launch_app(dataset, height=600, auto=True)